This notebook goes through the steps of training the RefDNN model, inclusing the preparation of the dataset. We take the CCLE dataset as an example here.

In [1]:
# import libraries
import os
import math
import numpy as np
import pandas as pd
import skopt
from datetime import datetime
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from refdnn import REFDNN

In [2]:
# key arguments

global outputdir
global checkpointdir
global verbose

# A filepath of drug response data for TRAINING
responseFile = '../data/response_CCLE.csv'          
# A filepath of gene expression data for TRAINING
expressionFile = '../data/expression_CCLE.csv'       
# A filepath of fingerprint data for TRAINING
fingerprintFile = '../data/fingerprint_CCLE.csv'      
# A directory path for saving outputs (default:'output_1')
outputdir = '../output_2_CCLE'         
# A size of batch on training process. The small size is recommended if an available size of RAM is small (default: 64)
batchsize = 32 #b        
# Number of training steps on training process. It is recommended that the steps is larger than (numpairs /batchsize) (default: 5000)
numtrainingsteps  = 1000 #t
# Number of bayesian search for hyperparameter tuning (default: 20)
numbayesiansearch = 10 #s
# K for outer k-fold cross validation (default: 5)
outerkfold = 5 # k        
# L for inner l-fold cross validation (default: 3)
innerkfold = 3 # l       
# 0:No logging, 1:Basic logging to check process, 2:Full logging for debugging (default:1)
verbose = 2 # v         

### Take a look at the input data files:

In [3]:
# A filepath of drug response data for TRAINING
response = pd.read_csv(responseFile)        
# A filepath of gene expression data for TRAINING
expression = pd.read_csv(expressionFile) 
# A filepath of fingerprint data for TRAINING
fingerprint = pd.read_csv(fingerprintFile) 

1. response data

In [4]:
response

,Drug,Cell,Label,LN_IC50
0,Nilotinib,22RV1_PROSTATE,Resistance,2.079442
1,Nilotinib,5637_URINARY_TRACT,Resistance,2.011612
2,Nilotinib,639V_URINARY_TRACT,Resistance,2.079442
3,Nilotinib,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,Sensitivity,0.647331
4,Nilotinib,769P_KIDNEY,Resistance,2.079442
...,...,...,...,...
5719,Erlotinib,WM88_SKIN,Resistance,2.079442
5720,Erlotinib,WM983B_SKIN,Resistance,2.079442
5721,Erlotinib,YKG1_CENTRAL_NERVOUS_SYSTEM,Resistance,2.079442
5722,Erlotinib,ZR751_BREAST,Resistance,2.079442


In [5]:
print("Number of drugs: ", len(set(response.Drug)))

Number of drugs:  12


In [6]:
print("Drugs: ", ', '.join(list(set(response.Drug)))) # the 12 drugs

Drugs:  Nutlin-3a (-), Paclitaxel, Nilotinib, Lapatinib, PD0325901, Palbociclib, Tanespimycin, PHA-665752, PLX-4720, Selumetinib, Sorafenib, Erlotinib


In [7]:
print("Number of cell lines: ", len(set(response.Cell)))

Number of cell lines:  491


In [8]:
print("Resistance response: ", len(response[response.Label=="Resistance"]))
print("Sensitive response: ", len(response[response.Label=="Sensitivity"]))

Resistance response:  3402
Sensitive response:  2322


2. gene expression profiles of cell lines

In [9]:
expression

,Unnamed: 0,Gene,1321N1_CENTRAL_NERVOUS_SYSTEM,22RV1_PROSTATE,42MGBA_CENTRAL_NERVOUS_SYSTEM,5637_URINARY_TRACT,639V_URINARY_TRACT,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,769P_KIDNEY,786O_KIDNEY,...,VMRCRCZ_KIDNEY,WM115_SKIN,WM1799_SKIN,WM2664_SKIN,WM793_SKIN,WM88_SKIN,WM983B_SKIN,YKG1_CENTRAL_NERVOUS_SYSTEM,ZR751_BREAST,ZR7530_BREAST
0,0,183,4.33,3.82,3.91,3.85,3.90,3.82,4.28,4.02,...,5.20,3.95,3.90,5.03,3.71,8.46,4.14,5.42,4.10,4.87
1,1,9383,4.35,4.08,4.49,4.62,4.20,4.59,6.81,4.38,...,7.27,4.04,4.26,4.23,4.28,5.86,4.32,8.27,7.80,7.43
2,2,221981,5.29,3.71,3.75,3.70,3.99,3.94,4.08,3.92,...,3.94,4.06,3.86,5.31,3.66,3.83,3.68,7.53,3.80,3.87
3,3,10451,4.30,7.41,5.12,5.78,4.21,9.37,10.28,4.55,...,7.91,4.29,5.64,4.35,4.44,7.53,4.66,5.76,8.79,7.56
4,4,100507322,3.32,3.52,3.62,3.53,3.27,4.96,3.67,3.59,...,3.62,3.53,3.39,3.24,3.36,3.42,3.43,3.37,3.91,4.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18921,18921,84616,3.49,3.38,3.73,3.56,3.31,3.88,3.62,3.47,...,3.66,3.44,3.46,3.54,3.32,3.87,3.64,3.38,3.40,3.56
18922,18922,4161,4.43,4.73,6.11,4.53,4.35,4.47,4.22,4.58,...,4.28,4.34,4.49,4.44,4.22,4.54,4.56,4.51,4.49,4.33
18923,18923,286527,8.23,8.04,8.80,10.49,8.83,9.97,7.21,8.49,...,6.48,7.82,8.20,8.86,5.84,5.01,8.30,9.60,8.32,5.35
18924,18924,10500,4.88,4.95,4.89,5.03,5.18,5.74,4.69,5.08,...,5.21,4.99,4.66,4.83,4.78,4.94,4.71,4.87,4.35,5.00


In [10]:
print(f"Expression data: {len(expression)} genes and {len(list(expression))-2} cell lines")

Expression data: 18926 genes and 491 cell lines


3. fingerprint data of drugs

In [11]:
fingerprint

,Name,FP1,FP2,FP3,FP4,FP5,FP6,FP7,FP8,FP9,...,FP3063,FP3064,FP3065,FP3066,FP3067,FP3068,FP3069,FP3070,FP3071,FP3072
0,Paclitaxel,0,0,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PHA-665752,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,Erlotinib,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Nutlin-3a (-),0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,PD0325901,0,0,1,0,0,1,0,0,0,...,1,0,1,1,0,0,0,0,0,0
5,Sorafenib,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Lapatinib,1,1,0,0,1,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0
7,Selumetinib,1,1,1,0,0,0,1,0,0,...,1,0,1,1,0,0,0,0,0,0
8,Nilotinib,0,1,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9,Palbociclib,0,1,1,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0


In [12]:
print(f"Fingerprint: {len(fingerprint)} drugs and {len(list(fingerprint))-1}-dimensional fingerprint representation")

Fingerprint: 12 drugs and 3072-dimensional fingerprint representation


## The main process of the Bayesian Optimization steps:

In [13]:
########################################################
## 0. Start
########################################################  

if verbose > 0:
    print('[START]')

if verbose > 1:
    print('[ARGUMENT] RESPONSEFILE: {}'.format(responseFile))
    print('[ARGUMENT] EXPRESSIONFILE: {}'.format(expressionFile))
    print('[ARGUMENT] FINGERPRINTFILE: {}'.format(fingerprintFile))
    print('[ARGUMENT] OUTPUTDIR: {}'.format(outputdir))
    print('[ARGUMENT] NUMBAYESIANSEARCH: {}'.format(numbayesiansearch))
    print('[ARGUMENT] OUTERKFOLD: {}'.format(outerkfold))
    print('[ARGUMENT] INNERKFOLD: {}'.format(innerkfold))
    print('[ARGUMENT] VERBOSE: {}'.format(verbose))

## output directory
if not os.path.exists(outputdir):
    os.mkdir(outputdir)
checkpointdir = os.path.join(outputdir, "checkpoint")
if not os.path.exists(checkpointdir):
    os.mkdir(checkpointdir)

[START]
[ARGUMENT] RESPONSEFILE: data/response_CCLE.csv
[ARGUMENT] EXPRESSIONFILE: data/expression_CCLE.csv
[ARGUMENT] FINGERPRINTFILE: data/fingerprint_CCLE.csv
[ARGUMENT] OUTPUTDIR: output_2_CCLE
[ARGUMENT] NUMBAYESIANSEARCH: 10
[ARGUMENT] OUTERKFOLD: 5
[ARGUMENT] INNERKFOLD: 3
[ARGUMENT] VERBOSE: 2


### The DATASET class:

In [14]:
class DATASET:
    def __init__(self, drfile, gefile, fpfile):
        ## 1. Read data
        ## 1-1) Gene expression
        self.ge = pd.read_csv(gefile, sep=',', index_col=0)
        ## 1-2) Drug response
        self.dr = pd.read_csv(drfile, sep=',', dtype='str')
        self.DRUGKEY = self.dr.columns[0]
        self.CELLKEY = self.dr.columns[1]
        self.LABELKEY = self.dr.columns[2]
        ## 1-3) Fingerprint
        self.fp = pd.read_csv(fpfile, sep=',', index_col=0).transpose()
        
        ## 2. Preprocessing
        ## 2-1) Find targets
        target_drugs = self._find_target_drugs()
        target_cells = self._find_target_cells()
        ## 2-2) Filter data
        self.ge = self.ge.filter(target_cells)
        self.dr = self.dr[self._get_target_idx(target_drugs, target_cells)]
        ## 2-3) Label string to integer
        idx = self.dr[self.LABELKEY] == 'Resistance'
        self.dr[self.LABELKEY][idx] = 1
        self.dr[self.LABELKEY][~idx] = 0
        self.dr[self.LABELKEY] = self.dr[self.LABELKEY].astype(np.uint8)
        
        ## 3. Structural Similarity Profile
        self.SSP = self._make_SSP()

    def __len__(self):
        return len(self.dr)
        
    def get_drugs(self, unique=False):
        return self._get_series(self.DRUGKEY, unique)
        
    def get_cells(self, unique=False):
        return self._get_series(self.CELLKEY, unique)
        
    def get_labels(self, unique=False):
        return self._get_series(self.LABELKEY, unique)
        
    def get_genes(self):
        return self.ge.index.values
        
    def get_exprs(self):
        return self.ge.values.T # per cell
    
    def make_xdata(self, idx=None):
        if idx is None:
            cells = self.get_cells()
        else:
            cells = self.get_cells()[idx]
        return np.array([self.ge[cell].values for cell in cells], dtype=np.float32)
    
    def make_ydata(self, idx=None):
        if idx is None:
            labels = self.get_labels()
        else:
            labels = self.get_labels()[idx]
        return np.expand_dims(np.array(labels, dtype=np.uint8), axis=-1)
    
    def make_sdata(self, base_drugs, idx=None):
        SSP = self.SSP.filter(base_drugs, axis='index')
        if idx is None:
            drugs = self.get_drugs()
        else:
            drugs = self.get_drugs()[idx]
        return np.array([SSP[drug].values for drug in drugs], dtype=np.float32)
        
    def make_idata(self, base_drugs, idx=None):
        drug2idx = {drug:i for i, drug in enumerate(base_drugs)}
        if idx is None:
            drugs = self.get_drugs()
        else:
            drugs = self.get_drugs()[idx]
        return np.array([drug2idx[drug] for drug in drugs], dtype=np.int64)
    
    def _make_SSP(self):
        ssp_mat = 1. - squareform(pdist(self.fp.values.T, 'jaccard'))
        return pd.DataFrame(ssp_mat, index=self.fp.columns, columns=self.fp.columns)
        
    def _get_series(self, KEY, unique):
        if unique:
            return np.sort(self.dr[KEY].unique(), kind='mergesort')
        else:
            return self.dr[KEY].values
    
    def _find_target_drugs(self):
        target_drugs = []
        self.drug2fp = {}
        for drugname in self.dr[self.DRUGKEY].unique():
            if drugname in self.fp:
                target_drugs.append(drugname)
                self.drug2fp[drugname] = self.fp[drugname].astype(np.uint8).astype(bool)
        return target_drugs
        
    def _find_target_cells(self):
        return self.ge.columns.intersection(self.dr[self.CELLKEY].unique())

    def _get_target_idx(self, target_drugs, target_cells):
        idx_drugs = self.dr[self.DRUGKEY].isin(target_drugs)
        idx_cells = self.dr[self.CELLKEY].isin(target_cells)
        return idx_drugs & idx_cells    

### 1. Read data

In [15]:
global dataset
dataset = DATASET(responseFile, expressionFile, fingerprintFile)
if verbose > 0:
    print('[DATA] NUM_PAIRS: {}'.format(len(dataset)))
    print('[DATA] NUM_DRUGS: {}'.format(len(dataset.get_drugs(unique=True))))
    print('[DATA] NUM_CELLS: {}'.format(len(dataset.get_cells(unique=True))))
    print('[DATA] NUM_GENES: {}'.format(len(dataset.get_genes())))
    print('[DATA] NUM_SENSITIVITY: {}'.format(np.count_nonzero(dataset.get_labels()==0)))
    print('[DATA] NUM_RESISTANCE: {}'.format(np.count_nonzero(dataset.get_labels()==1)))

## time log
timeformat = '[TIME] [{0}] {1.year}-{1.month}-{1.day} {1.hour}:{1.minute}:{1.second}'
if verbose > 0:
    print(timeformat.format(1, datetime.now()))

[DATA] NUM_PAIRS: 5724
[DATA] NUM_DRUGS: 12
[DATA] NUM_CELLS: 491
[DATA] NUM_GENES: 18926
[DATA] NUM_SENSITIVITY: 2322
[DATA] NUM_RESISTANCE: 3402
[TIME] [1] 2022-4-13 5:40:25


### 2. Define the space of hyperparameters

In [16]:
## 2-1) Set the range of hyperparameters
space_hidden_units = skopt.space.Integer(low=4, high=128, name='hidden_units')
space_learning_rate_ftrl = skopt.space.Real(low=1e-6, high=1e-1, prior='log-uniform', name='learning_rate_ftrl')
space_learning_rate_adam = skopt.space.Real(low=1e-6, high=1e-1, prior='log-uniform', name='learning_rate_adam')
space_l1_regularization_strength = skopt.space.Real(low=1e-3, high=1e+2, prior='log-uniform', name='l1_regularization_strength')
space_l2_regularization_strength = skopt.space.Real(low=1e-3, high=1e+2, prior='log-uniform', name='l2_regularization_strength')

## 2-2) Define hyperparmeter space
dimensions_hyperparameters = [space_hidden_units,
                              space_learning_rate_ftrl,
                              space_learning_rate_adam,
                              space_l1_regularization_strength,
                              space_l2_regularization_strength]

## time log
if verbose > 0:
    print(timeformat.format(2, datetime.now()))

[TIME] [2] 2022-4-13 5:40:25


### Define this fitness function for the hyperparameter tuning process:

In [17]:
def fitness(hyperparameters):
    global outputdir
    global checkpointdir
    global verbose
    global dataset
    global fitness_step
    global fitness_idx_train
    global fitness_idx_test
    global innerkfold
    global batchsize
    global numtrainingsteps
    
    ## 1. Hyperparameters
    HIDDEN_UNITS = hyperparameters[0]
    LEARNING_RATE_FTRL = hyperparameters[1]
    LEARNING_RATE_ADAM = hyperparameters[2]
    L1_REGULARIZATION_STRENGTH = hyperparameters[3]
    L2_REGULARIZATION_STRENGTH = hyperparameters[4]
    
    ## 2. 2-fold Cross Validation
    if verbose > 1:
        print('[INNER] [{}/{}] NOW EVALUATING PARAMETERS IN THE INNER LOOP...'.format(fitness_step, innerkfold))
        
    objective_metrics = 0.
    kf = StratifiedKFold(n_splits=innerkfold, shuffle=True)
    for k, (idx_construction, idx_validation) in enumerate(
        kf.split(X=np.zeros_like(fitness_idx_train), y=dataset.get_drugs()[fitness_idx_train])):
        
        ## 2-1) dataset
        idx_construction = fitness_idx_train[idx_construction]
        idx_validation = fitness_idx_train[idx_validation]
        base_drugs = np.unique(dataset.get_drugs()[idx_construction])
        
        X_construction = dataset.make_xdata(idx_construction)
        S_construction = dataset.make_sdata(base_drugs, idx_construction)
        I_construction = dataset.make_idata(base_drugs, idx_construction)
        Y_construction = dataset.make_ydata(idx_construction)
        
        X_validation = dataset.make_xdata(idx_validation)
        S_validation = dataset.make_sdata(base_drugs, idx_validation)
        I_validation = dataset.make_idata(base_drugs, idx_validation)
        Y_validation = dataset.make_ydata(idx_validation)
        
        ## 2-2) Create a model
        checkpoint_path = "RefDNN_cv_inner.ckpt"
        checkpoint_path = os.path.join(checkpointdir, checkpoint_path)
        clf = REFDNN(hidden_units=HIDDEN_UNITS,
                     learning_rate_ftrl=LEARNING_RATE_FTRL,
                     learning_rate_adam=LEARNING_RATE_ADAM,
                     l1_regularization_strength=L1_REGULARIZATION_STRENGTH,
                     l2_regularization_strength=L2_REGULARIZATION_STRENGTH,
                     batch_size=batchsize,
                     training_steps=numtrainingsteps,
                     checkpoint_path=checkpoint_path)
                    
        ## 2-3) Fit a model
        history = clf.fit(X_construction, S_construction, I_construction, Y_construction,
                          X_validation, S_validation, I_validation, Y_validation,
                          verbose=verbose)
        
        ## 2-4) Compute the metric
        Pred_validation = clf.predict(X_validation, S_validation, verbose=verbose)
        objective_metrics += accuracy_score(Y_validation, Pred_validation)
    
    training_accuracy = objective_metrics / kf.get_n_splits()
    
    if verbose > 1:
        print('[INNER] [{}/{}] HIDDEN_UNITS: {}'.format(fitness_step, innerkfold, HIDDEN_UNITS))
        print('[INNER] [{}/{}] LEARNING_RATE_FTRL: {:.3e}'.format(fitness_step, innerkfold, LEARNING_RATE_FTRL))
        print('[INNER] [{}/{}] LEARNING_RATE_ADAM: {:.3e}'.format(fitness_step, innerkfold, LEARNING_RATE_ADAM))
        print('[INNER] [{}/{}] L1_REGULARIZATION_STRENGTH: {:.3e}'.format(fitness_step, innerkfold, L1_REGULARIZATION_STRENGTH))
        print('[INNER] [{}/{}] L2_REGULARIZATION_STRENGTH: {:.3e}'.format(fitness_step, innerkfold, L2_REGULARIZATION_STRENGTH))
        print('[INNER] [{}/{}] TRAINING_ACCURACY: {:.3f}'.format(fitness_step, innerkfold, training_accuracy))
    
    fitness_step += 1
    return -training_accuracy

###  3. Start the hyperparameter tuning jobs

Note that this is still the old version of the script, so there are some warnnings in the beginning of the log, which doesn't affect the result. Also the labels of the loops also have some problems, which were already fixed in the updated version of the scripts (see the scripts folder).

In [18]:
global fitness_step
global fitness_idx_train
global fitness_idx_test
global innerkfold
global batchsize
global numtrainingsteps
    
## 3-1) init lists for metrics
ACCURACY_outer = []
AUCROC_outer = []
AUCPR_outer = []

## 3-2) init lists for hyperparameters
Hidden_units_outer = []
Learning_rate_ftrl_outer = []
Learning_rate_adam_outer = []
L1_strength_outer = []
L2_strength_outer = []

kf = StratifiedKFold(n_splits=outerkfold, shuffle=True)
for k, (idx_train, idx_test) in enumerate(kf.split(X=np.zeros(len(dataset)), y=dataset.get_drugs())):        
    fitness_step = 1
    fitness_idx_train = idx_train
    fitness_idx_test = idx_test

    ## 3-3) Bayesian optimization with gaussian process
    if verbose > 0:
        print('[OUTER] [{}/{}] NOW TUNING THE MODEL USING BAYESIAN OPTIMIZATION...'.format(k+1, kf.get_n_splits()))

    search_result = skopt.gp_minimize(func=fitness,
                                      dimensions=dimensions_hyperparameters,
                                      n_calls=numbayesiansearch,
                                      n_random_starts=1,
                                      acq_func='EI',
                                      noise=1e-10,
                                      verbose=0)
    
    BEST_HIDDEN_UNITS = search_result.x[0]
    BEST_LEARNING_RATE_FTRL = search_result.x[1]
    BEST_LEARNING_RATE_ADAM = search_result.x[2]
    BEST_L1_REGULARIZATION_STRENGTH = search_result.x[3]
    BEST_L2_REGULARIZATION_STRENGTH = search_result.x[4]
    BEST_TRAINING_ACCURACY = search_result.fun

    Hidden_units_outer.append(BEST_HIDDEN_UNITS)
    Learning_rate_ftrl_outer.append(BEST_LEARNING_RATE_FTRL)
    Learning_rate_adam_outer.append(BEST_LEARNING_RATE_ADAM)
    L1_strength_outer.append(BEST_L1_REGULARIZATION_STRENGTH)
    L2_strength_outer.append(BEST_L2_REGULARIZATION_STRENGTH)

    if verbose > 0:
        print('[OUTER] [{}/{}] BEST_HIDDEN_UNITS : {}'.format(k+1, kf.get_n_splits(), BEST_HIDDEN_UNITS))
        print('[OUTER] [{}/{}] BEST_LEARNING_RATE_FTRL : {:.3e}'.format(k+1, kf.get_n_splits(), BEST_LEARNING_RATE_FTRL))
        print('[OUTER] [{}/{}] BEST_LEARNING_RATE_ADAM : {:.3e}'.format(k+1, kf.get_n_splits(), BEST_LEARNING_RATE_ADAM))
        print('[OUTER] [{}/{}] BEST_L1_REGULARIZATION_STRENGTH : {:.3e}'.format(k+1, kf.get_n_splits(), BEST_L1_REGULARIZATION_STRENGTH))
        print('[OUTER] [{}/{}] BEST_L2_REGULARIZATION_STRENGTH : {:.3e}'.format(k+1, kf.get_n_splits(), BEST_L2_REGULARIZATION_STRENGTH))
        print('[OUTER] [{}/{}] BEST_TRAINING_ACCURACY : {:.3f}'.format(k+1, kf.get_n_splits(), BEST_TRAINING_ACCURACY))

    ## 3-4) Dataset
    idx_train_train, idx_train_valid = train_test_split(idx_train, test_size=0.2, stratify=dataset.get_drugs()[idx_train])
    base_drugs = np.unique(dataset.get_drugs()[idx_train_train])

    X_train = dataset.make_xdata(idx_train_train)
    S_train = dataset.make_sdata(base_drugs, idx_train_train)
    I_train = dataset.make_idata(base_drugs, idx_train_train)
    Y_train = dataset.make_ydata(idx_train_train)

    X_valid = dataset.make_xdata(idx_train_valid)
    S_valid = dataset.make_sdata(base_drugs, idx_train_valid)
    I_valid = dataset.make_idata(base_drugs, idx_train_valid)
    Y_valid = dataset.make_ydata(idx_train_valid)

    X_test = dataset.make_xdata(idx_test)
    S_test = dataset.make_sdata(base_drugs, idx_test)
    Y_test = dataset.make_ydata(idx_test)

    ## 3-5) Create a model using the best parameters
    if verbose > 0:
        print('[OUTER] [{}/{}] NOW TRAINING THE MODEL WITH BEST PARAMETERS...'.format(k+1, kf.get_n_splits()))

    checkpoint_path = "RefDNN_cv_outer.ckpt"
    checkpoint_path = os.path.join(checkpointdir, checkpoint_path)
    clf = REFDNN(hidden_units=BEST_HIDDEN_UNITS,
                 learning_rate_ftrl=BEST_LEARNING_RATE_FTRL,
                 learning_rate_adam=BEST_LEARNING_RATE_ADAM,
                 l1_regularization_strength=BEST_L1_REGULARIZATION_STRENGTH,
                 l2_regularization_strength=BEST_L2_REGULARIZATION_STRENGTH,
                 batch_size=batchsize,
                 training_steps=numtrainingsteps,
                 checkpoint_path=checkpoint_path)

    ## 3-6) Fit a model
    history = clf.fit(X_train, S_train, I_train, Y_train,
                      X_valid, S_valid, I_valid, Y_valid,
                      verbose=verbose)

    ## 3-7) Compute the metric
    Pred_test = clf.predict(X_test, S_test, verbose=verbose)
    Prob_test = clf.predict_proba(X_test, S_test, verbose=verbose)

    ACCURACY_outer_k = accuracy_score(Y_test, Pred_test)
    ACCURACY_outer.append(ACCURACY_outer_k)

    AUCROC_outer_k = roc_auc_score(Y_test, Prob_test)
    AUCROC_outer.append(AUCROC_outer_k)

    AUCPR_outer_k = average_precision_score(Y_test, Prob_test)
    AUCPR_outer.append(AUCPR_outer_k)

    if verbose > 0:
        print('[OUTER] [{}/{}] BEST_TEST_ACCURACY : {:.3f}'.format(k+1, kf.get_n_splits(), ACCURACY_outer_k))
        print('[OUTER] [{}/{}] BEST_TEST_AUCROC : {:.3f}'.format(k+1, kf.get_n_splits(), AUCROC_outer_k))
        print('[OUTER] [{}/{}] BEST_TEST_AUCPR : {:.3f}'.format(k+1, kf.get_n_splits(), AUCPR_outer_k))

    ## time log   
    if verbose > 0:
        print(timeformat.format(3, datetime.now()))

[OUTER] [1/5] NOW TUNING THE MODEL USING BAYESIAN OPTIMIZATION...
[INNER] [1/3] NOW EVALUATING PARAMETERS IN THE INNER LOOP...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.compat.v1.data.get_output_types(dataset)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
[RefDNN] [00100] LOSS_train=13.27263 | LOSS_valid=40.16729 | ACC_valid=0.344 | PRECISION_valid=0.000 | RECALL_valid=0.000 | (2.627 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCL

[INNER] [2/3] NOW EVALUATING PARAMETERS IN THE INNER LOOP...
[RefDNN] [00100] LOSS_train=0.37633 | LOSS_valid=0.74657 | ACC_valid=0.353 | PRECISION_valid=0.830 | RECALL_valid=0.018 | (1.413 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00200] LOSS_train=0.51683 | LOSS_valid=0.66965 | ACC_valid=0.491 | PRECISION_valid=0.944 | RECALL_valid=0.169 | (1.060 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00300] LOSS_train=0.43691 | LOSS_valid=0.55737 | ACC_valid=0.589 | PRECISION_valid=0.935 | RECALL_valid=0.340 | (0.879 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00400] LOSS_train=0.36767 | LOSS_valid=0.45974 | ACC_valid=0.655 | PRECISION_valid=0.926 | RECALL_valid=0.461 | (0.869 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00500] LOSS_train=0.51245 | LOSS_valid=0.4525

[RefDNN] [00700] LOSS_train=0.32968 | LOSS_valid=0.30682 | ACC_valid=0.705 | PRECISION_valid=0.675 | RECALL_valid=0.992 | (0.867 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00800] LOSS_train=0.39010 | LOSS_valid=0.29073 | ACC_valid=0.733 | PRECISION_valid=0.698 | RECALL_valid=0.990 | (0.867 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00900] LOSS_train=0.24036 | LOSS_valid=0.28132 | ACC_valid=0.755 | PRECISION_valid=0.716 | RECALL_valid=0.988 | (0.867 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [01000] LOSS_train=0.20590 | LOSS_valid=0.27473 | ACC_valid=0.772 | PRECISION_valid=0.731 | RECALL_valid=0.987 | (0.865 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
INFO:tensorflow:Restoring parameters from output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [CHECKPOINT] Model 

[RefDNN] [CHECKPOINT] Model is restored from: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00100] LOSS_train=0.50218 | LOSS_valid=0.67403 | ACC_valid=0.706 | PRECISION_valid=0.953 | RECALL_valid=0.558 | (1.384 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00200] LOSS_train=0.28469 | LOSS_valid=0.47041 | ACC_valid=0.587 | PRECISION_valid=0.755 | RECALL_valid=0.499 | (0.910 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00300] LOSS_train=0.26536 | LOSS_valid=0.43056 | ACC_valid=0.589 | PRECISION_valid=0.643 | RECALL_valid=0.706 | (0.873 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00400] LOSS_train=0.18852 | LOSS_valid=0.36775 | ACC_valid=0.617 | PRECISION_valid=0.647 | RECALL_valid=0.786 | (0.849 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00500] LOSS_tr

INFO:tensorflow:Restoring parameters from output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [CHECKPOINT] Model is restored from: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00100] LOSS_train=0.47842 | LOSS_valid=0.51216 | ACC_valid=0.905 | PRECISION_valid=0.908 | RECALL_valid=0.937 | (1.735 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00200] LOSS_train=0.52166 | LOSS_valid=0.49432 | ACC_valid=0.906 | PRECISION_valid=0.890 | RECALL_valid=0.964 | (0.903 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00300] LOSS_train=0.57605 | LOSS_valid=0.49105 | ACC_valid=0.904 | PRECISION_valid=0.886 | RECALL_valid=0.965 | (0.858 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00400] LOSS_train=0.35713 | LOSS_valid=0.67034 | ACC_valid=0.869 | PRECISION_valid=0.839 | RECALL_valid=0.970 | (0.856 sec)
[RefDNN] [00500] LOSS_t

...

[RefDNN] [CHECKPOINT] Model is restored from: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00100] LOSS_train=0.51594 | LOSS_valid=0.99262 | ACC_valid=0.717 | PRECISION_valid=0.695 | RECALL_valid=0.979 | (1.732 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00200] LOSS_train=0.65705 | LOSS_valid=0.86749 | ACC_valid=0.728 | PRECISION_valid=0.691 | RECALL_valid=0.991 | (0.927 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00300] LOSS_train=0.41427 | LOSS_valid=0.76179 | ACC_valid=0.791 | PRECISION_valid=0.747 | RECALL_valid=0.979 | (0.881 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00400] LOSS_train=0.48884 | LOSS_valid=0.67295 | ACC_valid=0.817 | PRECISION_valid=0.777 | RECALL_valid=0.968 | (0.883 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00500] LOSS_tr

[RefDNN] [00500] LOSS_train=0.54526 | LOSS_valid=0.53678 | ACC_valid=0.586 | PRECISION_valid=0.667 | RECALL_valid=0.633 | (0.878 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00600] LOSS_train=0.53325 | LOSS_valid=0.51408 | ACC_valid=0.624 | PRECISION_valid=0.688 | RECALL_valid=0.695 | (0.878 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00700] LOSS_train=0.44431 | LOSS_valid=0.49359 | ACC_valid=0.658 | PRECISION_valid=0.709 | RECALL_valid=0.737 | (0.879 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00800] LOSS_train=0.41816 | LOSS_valid=0.47814 | ACC_valid=0.683 | PRECISION_valid=0.725 | RECALL_valid=0.768 | (0.879 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00900] LOSS_train=0.36978 | LOSS_valid=0.46509 | ACC_valid=0.703 | PRECISION_valid=0.737 | RECALL_valid=0.

[RefDNN] [01000] LOSS_train=0.07779 | LOSS_valid=0.33673 | ACC_valid=0.771 | PRECISION_valid=0.725 | RECALL_valid=0.983 | (0.848 sec)
INFO:tensorflow:Restoring parameters from output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [CHECKPOINT] Model is restored from: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00100] LOSS_train=0.46402 | LOSS_valid=0.69384 | ACC_valid=0.640 | PRECISION_valid=0.640 | RECALL_valid=0.979 | (1.401 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00200] LOSS_train=0.53643 | LOSS_valid=0.65548 | ACC_valid=0.608 | PRECISION_valid=0.608 | RECALL_valid=1.000 | (0.917 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00300] LOSS_train=0.50157 | LOSS_valid=0.58951 | ACC_valid=0.625 | PRECISION_valid=0.617 | RECALL_valid=1.000 | (0.862 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00400] LOSS_t

[RefDNN] [00500] LOSS_train=0.79959 | LOSS_valid=0.64816 | ACC_valid=0.672 | PRECISION_valid=0.643 | RECALL_valid=0.994 | (0.886 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00600] LOSS_train=0.71002 | LOSS_valid=0.59308 | ACC_valid=0.712 | PRECISION_valid=0.673 | RECALL_valid=0.990 | (0.884 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00700] LOSS_train=0.34256 | LOSS_valid=0.58120 | ACC_valid=0.740 | PRECISION_valid=0.696 | RECALL_valid=0.987 | (0.886 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00800] LOSS_train=0.68382 | LOSS_valid=0.56094 | ACC_valid=0.760 | PRECISION_valid=0.714 | RECALL_valid=0.985 | (0.880 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [00900] LOSS_train=0.41650 | LOSS_valid=0.54174 | ACC_valid=0.776 | PRECISION_valid=0.729 | RECALL_valid=0.

[RefDNN] [00900] LOSS_train=0.21057 | LOSS_valid=0.31628 | ACC_valid=0.769 | PRECISION_valid=0.730 | RECALL_valid=0.972 | (0.858 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [01000] LOSS_train=0.28562 | LOSS_valid=0.31618 | ACC_valid=0.782 | PRECISION_valid=0.743 | RECALL_valid=0.970 | (0.854 sec)
[RefDNN] [CHECKPOINT] Model is saved in: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
INFO:tensorflow:Restoring parameters from output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[RefDNN] [CHECKPOINT] Model is restored from: output_2_CCLE/checkpoint/RefDNN_cv_inner.ckpt
[INNER] [10/3] HIDDEN_UNITS: 94
[INNER] [10/3] LEARNING_RATE_FTRL: 7.312e-05
[INNER] [10/3] LEARNING_RATE_ADAM: 8.865e-05
[INNER] [10/3] L1_REGULARIZATION_STRENGTH: 2.924e+01
[INNER] [10/3] L2_REGULARIZATION_STRENGTH: 1.000e+02
[INNER] [10/3] TRAINING_ACCURACY: 0.897
[OUTER] [5/5] BEST_HIDDEN_UNITS : 128
[OUTER] [5/5] BEST_LEARNING_RATE_FTRL : 9.395e-02
[OUTER] [5/5] BEST_LEA

### 4. Save the results

In [19]:
res = pd.DataFrame.from_dict({'ACCURACY':ACCURACY_outer,
                              'AUCROC':AUCROC_outer,
                              'AUCPR':AUCPR_outer,
                              'Hidden_units':Hidden_units_outer,
                              'Learning_rate_ftrl':Learning_rate_ftrl_outer,
                              'Learning_rate_adam':Learning_rate_adam_outer,
                              'L1_regularization_strength':L1_strength_outer,
                              'L2_regularization_strength':L2_strength_outer})

res = res[['ACCURACY', 'AUCROC', 'AUCPR', 'Hidden_units', 'Learning_rate_ftrl', 'Learning_rate_adam', 'L1_regularization_strength', 'L2_regularization_strength']]
res.to_csv(os.path.join(outputdir, 'metrics_hyperparameters.csv'), sep=',')

## time log    
if verbose > 0:
    print(timeformat.format(4, datetime.now()))

if verbose > 0:
    print('[FINISH]')
    

[TIME] [4] 2022-4-13 6:13:58
[FINISH]


The whole process took 33 min in total (CCLE data, fast mode).

In [21]:
pd.read_csv("output_2_CCLE/metrics_hyperparameters.csv")

,Unnamed: 0,ACCURACY,AUCROC,AUCPR,Hidden_units,Learning_rate_ftrl,Learning_rate_adam,L1_regularization_strength,L2_regularization_strength
0,0,0.896070,0.943032,0.954028,32,0.022476,0.006480,5.297726,0.029783
1,1,0.910044,0.951772,0.963393,4,0.012673,0.023512,2.716970,0.022271
2,2,0.909170,0.932311,0.940374,58,0.000413,0.000076,1.646717,0.001137
3,3,0.910044,0.944293,0.952139,95,0.000095,0.100000,0.359450,0.001045
4,4,0.899476,0.933211,0.937594,128,0.093948,0.001286,100.000000,1.419744
